In [134]:
import pandas as pd
import dask.dataframe as dd
import modin.pandas as mpd
import time
import ray
import numpy as np

# Import datasets using pandas, dask and modin

In [9]:
# import data using pandas
train_start = time.time()
df_trian = pd.DataFrame(pd.read_csv('train.csv'))
train_end = time.time()
print('The time for importing trainning data is:',train_end - train_start)

test_start = time.time()
df_test = pd.read_csv('test.csv')
test_end = time.time()
print('The time for importing trainning data is:',test_end - test_start)

MemoryError: Unable to allocate 32.0 KiB for an array with shape (4096,) and data type int64

In [137]:
# using dask
train_start = time.time()
df_train = dd.read_csv('train.csv')
train_end = time.time()
print('The time for importing trainning data is:',train_end - train_start)

test_start = time.time()
df_test = dd.read_csv('test.csv')
test_end = time.time()
print('The time for importing testing data is:',test_end - test_start)

The time for importing trainning data is: 0.015405893325805664
The time for importing testing data is: 0.01587057113647461


In [140]:
df_train = df_train.drop('Unnamed: 0',axis = 1)
df_test = df_test.drop('Unnamed: 0',axis = 1)

In [141]:
def eda(df):
    print(df.head(2),'\n')
    print(df.info())
    print(df.describe(),'\n')
    print(df.isna().sum())
    print('Number of data point is: ', len(df))



### trian utility file

In [142]:
%%writefile trainutility.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re

#path = 'C:\Users\shuny\OneDrive\Desktop\Data Glacier\Week 6\train_file.yamlm'
def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string


def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0



Overwriting trainutility.py


### Train YAML file

In [144]:
type(df_train)

dask.dataframe.core.DataFrame

In [145]:
col_name = list(np.array(df_train.columns)[:6])

In [146]:
train_df_new = df_train[col_name]
train_df_new = train_df_new.head(20)
train_df_new.to_csv('./truncated_train.csv')


In [158]:
train_df_new.columns

Index(['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2'], dtype='object')

In [159]:
%%writefile train_file.yaml
file_type: csv
dataset_name: train_df_new
file_name: truncated_train
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - customer_ID
    - S_2
    - P_2
    - D_39
    - B_1
    - B_2

Overwriting train_file.yaml


In [160]:
import trainutility as train_util
config_data = train_util.read_config_file("train_file.yaml")

In [161]:
config_data['inbound_delimiter']

','

In [162]:
config_data

{'file_type': 'csv',
 'dataset_name': 'train_df_new',
 'file_name': 'truncated_train',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2']}

In [163]:
# read the file using config file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'
df = pd.read_csv(source_file,config_data['inbound_delimiter'])
df = df.drop('Unnamed: 0',axis = 1)
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0,0.008724,1.006838
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0,0.004923,1.000653
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,3,0.021655,1.009672
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0,0.013683,1.002700
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0,0.015193,1.000727


In [164]:
config_data

{'file_type': 'csv',
 'dataset_name': 'train_df_new',
 'file_name': 'truncated_train',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2']}

In [165]:
train_util.col_header_val(df,config_data)

column name and column length validation passed


1

In [166]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,config_data['columns'])

columns of files are: Index(['customer_id', 's_2', 'p_2', 'd_39', 'b_1', 'b_2'], dtype='object')
columns of YAML are: ['customer_ID', 'S_2', 'P_2', 'D_39', 'B_1', 'B_2']


In [169]:
if train_util.col_header_val(df,config_data)==0:
    print("validation failed")
else:
    print("col validation passed")

column name and column length validation passed
col validation passed
